Karla Jacobo  
DSCI 8950  
EDA Assignment

#### **Importing necessary packages and reading in datasets**
To start off the Exploratory Data Analysis, I am going to import the data from my team's selected data sources and store them in local variables for easier access. 

In [10]:

import numpy as np
import pandas as pd
import collections
import plotly.express as px
import plotly.graph_objects as go
import country_converter as coco

airline = pd.read_csv('../data/openFlightsRaw/airline.csv')
airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
countries = pd.read_csv('../data/openFlightsRaw/countries.csv')
planes = pd.read_csv('../data/openFlightsRaw/planes.csv')
routes = pd.read_csv('../data/openFlightsRaw/routes.csv')
routesDist = pd.read_csv('../data/routesDist.csv')

#### **Data Inspection**
In the next block of code, I am going to gather the following information from the datasets:
    - Number of Rows
    - Number of Columns
    - Data types

In [2]:
tables = [airline, airports, countries, planes, routes]
allTableTypes = []
airline.name = 'Airline'
airports.name = 'Airports'
countries.name = 'Countries'
planes.name = 'Planes'
routes.name = 'Routes'

for table in tables:
    print(f'{table.name} table size: {str(table.shape[0])} rows, {str(table.shape[1])} columns')

print()

for table in tables:
    dataTypes = table.dtypes.values

    for type in dataTypes:
        allTableTypes.append(str(np.dtype(type)))

print('All unique data types featured in tables: ')
print(collections.Counter(allTableTypes))


Airline table size: 6162 rows, 8 columns
Airports table size: 7698 rows, 14 columns
Countries table size: 261 rows, 3 columns
Planes table size: 246 rows, 3 columns
Routes table size: 67663 rows, 9 columns

All unique data types featured in tables: 
Counter({'object': 31, 'int64': 4, 'float64': 2})


Based on the information extracted above, we will be working with Integers, Floats, and Strings. For additional context, the dtype attribute labels Strings as Objects.

By including the counts of the unique data types in the tables, we can extrapolate that a majority of the data we have is categorical. With 31 columns being Strings, and the other 6 being Integers and Floats, the numerical data we have is slim.

Next I am going to find rows and columns within the data sets that contain null values.

In [3]:
for table in tables:
    null_data = table[table.isnull().any(axis=1)]
    null_data_by_column = table.isnull().sum().sort_values(ascending=False).head(3)

    print(f'{table.name} table: ', end='')
    print(f'{str(null_data.shape[0])} rows out of {str(table.shape[0])} are missing data. Columns with most data missing: ')

    for index, value in null_data_by_column.items():
        print(f'\t Column {str(index)} is missing {str(round(value/table.shape[0]*100, 2))}% it\'s data ({value}/{table.shape[0]}).')
    print()


Airline table: 5152 rows out of 6162 are missing data. Columns with most data missing: 
	 Column IATA is missing 75.09% it's data (4627/6162).
	 Column CLSGN is missing 13.11% it's data (808/6162).
	 Column ALIAS is missing 8.21% it's data (506/6162).

Airports table: 49 rows out of 7698 are missing data. Columns with most data missing: 
	 Column CTY is missing 0.64% it's data (49/7698).
	 Column AIRPT_ID is missing 0.0% it's data (0/7698).
	 Column NAME is missing 0.0% it's data (0/7698).

Countries table: 2 rows out of 261 are missing data. Columns with most data missing: 
	 Column ISO_CD is missing 0.38% it's data (1/261).
	 Column DAFIF_CD is missing 0.38% it's data (1/261).
	 Column NAME is missing 0.0% it's data (0/261).

Planes table: 0 rows out of 246 are missing data. Columns with most data missing: 
	 Column NAME is missing 0.0% it's data (0/246).
	 Column IATA is missing 0.0% it's data (0/246).
	 Column ICAO is missing 0.0% it's data (0/246).

Routes table: 53066 rows out of

Through the analysis conducted above, we can see that there are many instances of missing data. While that is not uncommon in data sets, getting more detail on where the data is missing will help narrow down any problem areas.

For example, the Airline table had the most missing data out of all of the tables. It seems as though one the IATA column is missing data for most of the rows in the table. Because we now know this, we can evaluate whether or not we need the data in that column.

#### **Data Cleaning**

Next, I am going to start cleaning some of the data. This will be done in two efforts:  
    &nbsp;&nbsp;&nbsp;&nbsp;1. Finding outliers in numerical data  
    &nbsp;&nbsp;&nbsp;&nbsp;2. Removing instances where large quantities of data are missing within single columns  
Given that we have six instances of numerical data in our datasets, there won't be many to go through when detecting numerical outliers.  


**Outlier Analysis**  
Based on my previous analysis above, we know that there are three unique data types in the datasets here: int64, float64, and objects. Only two of those are numerical. Because of that, I am going to iterate through every table and column and conduct an IQR analysis on those numerical values.

In [4]:
for table in tables:
    for column in table.columns:
        if not column.__contains__('ID') and not column.__contains__('LAT') and not column.__contains__('LON'):
            column_data = table[column]
            if column_data.dtype == 'int64' or column_data.dtype == 'float64':
                q1,q3 = np.percentile(column_data, [25,75])
                iqr = q3-q1
                boundary_value = 1.5
                outliers = (column_data < q1 - boundary_value * iqr) | (column_data > q3 + boundary_value * iqr)

                print(f'The following "outliers" have been detected in table {table.name}, column {column}: \n{column_data[outliers]}\n')

The following "outliers" have been detected in table Airports, column ALT: 
0       5282
2       5388
51      3043
111     2968
115     3048
        ... 
7668    5741
7678    3810
7682    4412
7684    7348
7685    3013
Name: ALT, Length: 813, dtype: int64

The following "outliers" have been detected in table Routes, column STOPS: 
2066     1
7810     1
8272     1
17767    1
25116    1
25220    1
25230    1
49746    1
63567    1
64042    1
64045    1
Name: STOPS, dtype: int64



Based on the analysis above, I would not call some of these columns true outliers. Initially, I ran an IQR analysis for all numerical values in the tables. However, I noticed a high reporting on numerical values for things that would not be affected by being out of that percentile range. For example, I made sure not to run an IQR analalysis for columns having to do with ID numbers, latitude, and longitude. Once removing those columns from the ones being analyzed, I was left with two sets of "outliers": flight altitude and number of stops. 

I plan to leave both sets of outliers in for the time being for the following reasons:  
&nbsp;&nbsp;&nbsp;&nbsp; 1. The outlying altitudes make up a large part of the airports dataset. By removing those, we may be removing valuable insight into the the routes going to and from those locations.  
&nbsp;&nbsp;&nbsp;&nbsp; 2. The outlying number of stops, being 1, could be the optimal number of stops for routes. By looking at those, we could discover what makes those routes most efficient.  

**Removal of Columns with Large Quantities of Missing Data:**
Because of the data inspection conducted above, we know which columns have the largest amount of missing data. The column missing the most data, with 75.09% of the rows being empty, is the IATA column in the Airline table. Because of that, I will be removing that column for the datagframe in the code below. Afterwards, I will be printing the names of the columns in the dataframe to show that the column is no longer there.

In [5]:
airline.drop('IATA', axis=1, inplace=True)

for column in airline:
    print(column)

AIRLINE_ID
NAME
ALIAS
ICAO
CLSGN
CTRY
ACTV


#### **Data Preprocessing**  
Because a majority of the data in our datasets is categorical, I will be focusing on preprocessing methods for that type of data. The following methods will be used for specific instances of the dataset:  
&nbsp;&nbsp;&nbsp;&nbsp; 1. Encoding will be used to better represent data in "Active" column on the "Airline" table. The data in this column is represented as a 'y' or 'n' to represent whether it is active or not. These text values will be converted to 1 for 'y' and 0 for 'n'.  
&nbsp;&nbsp;&nbsp;&nbsp; 2. I will be aggregating the CTRY column the Airline table to see which countries have the most variety of active airlines. One thing I noticed while aggregating the data and factoring in whether an airline was active or not, was that inactive airlines makeup a lot of the data in the Airline table. So much so, that we can see the top three countries with the highest airline counts shift below.

In [6]:
#Encoding the Active column in the Airline table

mapping = {'Y': 1, 'N': 0}
airline['ACTV'].replace(mapping, inplace=True)

print('\033[1mNew data in active column for Airline table: \033[0m')
print(airline['ACTV'].head(3))

print()
print('------------------------------------------------------------')
print()

#Aggregating Country data to show which has the most active airlines
active_airlines = airline.where(airline['ACTV'] != 0)
active_airline_ctry = active_airlines.groupby('CTRY').count()[['AIRLINE_ID']]
active_airline_ctry.reset_index(inplace=True)

airline_ctry = airline.groupby('CTRY').count()[['AIRLINE_ID']]
airline_ctry.reset_index(inplace=True)
print('\033[1mSample of Aggegate Airline data by Country\033[0m')
print(airline_ctry.sort_values(by='AIRLINE_ID', ascending=False).head(3))

print()
print('\033[1mSample of Aggegate Active Airline data by Country\033[0m')
print(active_airline_ctry.sort_values(by='AIRLINE_ID', ascending=False).head(3))

New data in active column for Airline table: 
0    1
1    1
2    0
Name: ACTV, dtype: object

------------------------------------------------------------

Sample of Aggegate Airline data by Country
               CTRY  AIRLINE_ID
265   United States        1099
176          Mexico         440
264  United Kingdom         414

Sample of Aggegate Active Airline data by Country
               CTRY  AIRLINE_ID
186   United States         156
146          Russia          80
185  United Kingdom          45


C:\Users\kayj4\AppData\Local\Temp\ipykernel_8960\1708732911.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airline['ACTV'].replace(mapping, inplace=True)


#### **Data Analysis and Visualizations**

For the Data Analysis and Visualization section, I will be utilizing some of the data that was preprocessed above.

**Countries with the most active airlines**  
For my first analysis and visualization, I will be depicting which countries have the most active airlines. Which will help indicate where most of the flow of the routes will be occuring. This should also help indicate which countries more likely involved in the routes.

Below we can see an interactive map depicting the counts of all of the countries in our Airline dataset. While creating this map, I also noticed that some of the entries in the countries column are not actually countries. This will be something I will mention to the team when we put our findings together.

In [7]:
active_airline_ctry['CTRY_CD'] = coco.convert(active_airline_ctry['CTRY'], to='ISO3') #Adding 3 letter country code to allow for mapping
active_airline_ctry['AIRLINE_COUNT'] = active_airline_ctry['AIRLINE_ID'] 

country_airlines_map = px.choropleth(active_airline_ctry, locations='CTRY_CD', color='AIRLINE_COUNT', title='Countries by Active Airline Count')
country_airlines_map.show()


 S.A. not found in regex
ALASKA not found in regex
AVIANCA not found in regex
Canadian Territories not found in regex
DRAGON not found in regex
Netherlands Antilles not found in regex
\N not found in ISO2


**Airport Plotting**

For this analysis and visualization, I will be plotting all of the airports on a map. In addition to that, I will be factoring in the frequency of it being a source airport or a destination airport. This visualization will utilize data from the Airports table and the Routes table. Once plotted, this will show which airports are the more popular hubs for routing.

Below, we can see the airports plotted on the map along with their frequency. The amount of times an airport has been involved in a route is captured by the size of the bubbles on the chart. By mapping it out this way, we can see which airports will be most involved in our routes.

In [8]:
routes_by_airport = routes.groupby('SRC_AIRPT_ID').count()[['SRC_AIRPT']]

routes_by_airport['SRC_AIRPT_COUNT'] = routes_by_airport['SRC_AIRPT'] #Renaming
routes_by_airport.drop('SRC_AIRPT', axis=1, inplace=True)
routes_by_airport.reset_index(inplace=True)

airports['AIRPT_ID'] = airports['AIRPT_ID'].apply(str)
routes_by_airport = routes_by_airport.join(airports.set_index('AIRPT_ID'), on='SRC_AIRPT_ID')
routes_by_airport.drop(['NAME', 'CTY', 'CTRY', 'IATA', 'ICAO', 'ALT', 'TZ', 'DST', 'DBTZ', 'TYPE', 'SRC'], axis=1, inplace=True) #Removing unnecessary fields

airports_by_freq = px.scatter_geo(routes_by_airport,
                     lat = routes_by_airport['LAT'],
                     lon = routes_by_airport['LONG'],
                     hover_name = routes_by_airport['SRC_AIRPT_ID'],
                     size= routes_by_airport['SRC_AIRPT_COUNT'],
                     projection="natural earth",
                     title = 'Airports By Route Frequency')

airports_by_freq.show()

### EDA Continued  

In [280]:
usRoutesDist = routesDist.where((routesDist["SRCCTRY"] == 'United States') & (routesDist["DESTINCTRY"] == 'United States'))
usRoutesDist = usRoutesDist.dropna()

routePlanes = usRoutesDist['EQPT'].astype(str)
routeStops = usRoutesDist['STOPS']
planeType = []
planeGroups = []

for planes in routePlanes:
    planeType += planes.split(' ')

    if (len(planes) > 3):
        planeGroups.append(tuple(planes.split(' ')))


planeGroupCount = collections.Counter(planeGroups)
stopsNumberCount = collections.Counter(routeStops)
planeTypeCount = collections.Counter(planeType)

planeGroupCount = pd.DataFrame.from_dict(planeGroupCount, orient='index').reset_index()
planeGroupCount = planeGroupCount.rename(columns={'index':'Type', 0:'Count'})
planeGroupCount = planeGroupCount.sort_values(by='Count', ascending=False)

planeTypeCount = pd.DataFrame.from_dict(planeTypeCount, orient='index').reset_index()
planeTypeCount = planeTypeCount.rename(columns={'index':'Type', 0:'Count'})
planeTypeCount['inGroupCount'] = None
planeTypeCount['alternatives'] = None
planeTypeCount['alternativeCount'] = None
planeTypeCount = planeTypeCount.sort_values(by='Count', ascending=False)

stopsNumberCount = pd.DataFrame.from_dict(stopsNumberCount, orient='index').reset_index()
stopsNumberCount = stopsNumberCount.rename(columns={'index':'Stops', 0:'Count'})

for i in range(len(planeTypeCount)):
    inGroupCount = 0
    group = []
    for j in range(len(planeGroupCount)):
        if planeTypeCount['Type'][i] in planeGroupCount['Type'][j]:
            inGroupCount += 1
            group += planeGroupCount['Type'][j]
    group = set(group)
    planeTypeCount.loc[i, "alternatives"] = str(group)
    planeTypeCount.loc[i, "alternativeCount"] = len(group)
    planeTypeCount.loc[i, "inGroupCount"] = inGroupCount

planeTypeCount['alternatives'] = planeTypeCount['alternatives'].replace("set()", None)
planeTypeCount = planeTypeCount.sort_values(by='Count', ascending=False)

display(planeGroupCount)
print('------------------------------------------------')
display(planeTypeCount)
print('------------------------------------------------')
display(stopsNumberCount)


,Type,Count
10,"(ER4, ERD)",90
52,"(ERD, ER4)",56
26,"(CR9, CRJ)",50
311,"(ERJ, CRJ)",46
9,"(CRJ, CR9)",45
...,...,...
338,"(E75, CRJ, ERJ)",1
337,"(73J, 73H, 734, 739, 73G)",1
336,"(CR9, CRJ, E75)",1
334,"(CR9, E75, E70)",1


------------------------------------------------


,Type,Count,inGroupCount,alternatives,alternativeCount
7,CRJ,1298,172,"{'CR9', 'E70', 'DH3', 'ER4', 'DH4', 'DH8', 'E9...",18
6,CR7,643,153,"{'DH3', 'ER4', '320', '73H', '738', 'DH2', 'DH...",23
22,ERJ,525,118,"{'CR9', 'E70', 'DH3', '320', 'ER4', 'DH4', 'E7...",15
28,73W,480,47,"{'CR9', '757', '320', '73W', '735', 'M90', '73...",16
9,ER4,434,72,"{'E70', 'DH3', 'ER4', 'DH4', 'E75', '738', 'ER...",11
26,M88,433,153,"{'CR9', '757', '320', '76W', '73W', 'M90', '73...",15
5,CR9,407,82,"{'CR9', 'E70', 'DH3', '320', 'DH8', '73W', 'E7...",16
24,757,280,100,"{'753', '757', '320', '76W', '763', 'M83', '73...",18
10,E75,270,63,"{'CR9', 'E70', 'ER4', '320', 'DH8', 'E90', '74...",14
11,E70,266,82,"{'CR9', 'E70', 'DH3', '320', 'DH4', 'ER4', 'E9...",18


------------------------------------------------


,Stops,Count
0,0.0,4623
